# Project description

With increasing renewable engergy generation and therefor rising fluctation of generated power, intelligent electricity grids are necessary for the balance between demand and the generation. The amount of feed-in power to the grid has to be at every time equal to the feed-out power, otherwise the grid gets unstable and can cause a black-out to a wider region. Also the sectors mobility, heating and industry are transforming there currently fossil-driven energy generators to electricity-based technologies, which will, despite of the conduction of energry efficiancy measures, lead to higher power demands in future.

This project uses a power plant register and measured time series of generation and consumption of Germany to provide a solid database for further electricity demand forecasts and simulations concerning different scenarios of the energetic transformation of our economies.

In this sample, the following sources are referenced:

1. Markstammdatenregister
Datenlizenz Deutschland – Namensnennung – Version 2.0 // http://www.govdata.de/dl-de/by-2-0
https://www.marktstammdatenregister.de/MaStR/Datendownload

2. Bundesnetzagentur | SMARD.de
https://www.smard.de/home/downloadcenter/download-marktdaten/

3. GeoJSON data Germany
https://github.com/yetzt/postleitzahlen/blob/main/data/postleitzahlen.geojson

# Getting started

Download current version of the data:

1. MaStR
	https://www.marktstammdatenregister.de/MaStR/Datendownload
	Download in section "Gesamtdatenauszug vom Vortag" the big ZIP-file and unpack it in a folder named "Gesamtdatenauszug_MaStR".


2. SMARD.de
	https://www.smard.de/home/downloadcenter/download-marktdaten/
	You need to download for each combination of "Oberkategorie" and "Datenkategorie" a csv-file. This code sample works for the following time series:
    
	Oberkategorie: Stromerzeugung // Datenkategorie: Realisierte Erzeugung
    
	Oberkategorie: Stromerzeugung // Datenkategorie: Prognostizierte Erzeugung
    
	Oberkategorie: Stromerverbrauch // Datenkategorie: Realisierter Stromverbrauch
    
	Oberkategorie: Stromerverbrauch // Datenkategorie: Prognostizierter Stromverbrauch
    
	Oberkategorie: Stromerzeugung // Datenkategorie: Installierte Erzeugerleistung

	
	Choose as "Land/Regelzone" the element "Land: Deutschland".
	
	Select the time period you want to analyse. If you want to download multiple years, best download for each year a csv.
	
	Choose "Auflösung wählen: Viertelstunde" and "Dateiformat: CSV". For the last combination "Oberkategorie: Stromerzeugung // Datenkategorie: Installierte Erzeugerleistung" choose "Auflösung: Tag".

	Save these files in a folder named "smard_historical_data".

	Copy the two folder "Gesamtdatenexport" and "Gesamtdatenauszug_MaStR" in a folder named "01_raw_data".

3. GeoJSON data Germany
	https://github.com/yetzt/postleitzahlen/blob/main/data/postleitzahlen.geojson

	"name": "postleitzahlen-de", "version": "1.0.2", "description": "german postcode areas 2023", "license": "ODbL-1.0",
	"contributors": [{
			"name": "yetzt",
			"url": "https://yetzt.me"
		},{
			"name": "OpenStreetMap contributors",
			"url": "https://www.openstreetmap.org/"
		}],
	"repository": {
		"type": "git",
		"url": "https://github.com/yetzt/postleitzahlen.git"}

4. folder structure
	Create somewhere on your computer a folder and create two folder in it (on the same level): "code_samples" and "code_samples_data"

	Create in each of these two folders an empty folder named "01_smard_grids"

	Copy the content of this repository in code_samples/01_smard_grids

	Copy the folder "01_raw_data" in code_samples_data/01_smard_grids

	Save postleitzahlen.geojson in the folder ..\\code_samples_data/01_smard_grids\\01_raw_data\\postcodes_geodata_Germany'

5. You will need to change some pathes in the code:

	common_namespace_SG.py : line 54 and 56

	app.py : line 5 (depending on your enviroment)

5. Continue with this notebook

# Set-up

In [ ]:
# Import functions from python file
import sys
#sys.path.append("..")
#path_to_modules = 'D:\\Coding\\code_samples\\smard_grids'
#sys.path.append(path_to_modules)
import common_namespace_SG as cn
import common_functions_SG as cf


# import other packages

import numpy as np
import pandas as pd
import os
from os.path import exists
import json
import pyproj
import time
#from datetime import datetime as dt
from datetime import datetime
from datetime import timedelta

import xml.etree.ElementTree as ET
import xmltodict

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import seaborn as sns
import seaborn.objects as so

#!pip install geopy
from geopy import distance
import math
#!pip install geopandas
import geopandas

#!pip install geojson'
import geojson
#!pip install geopandas
import geopandas

# Import Meteostat library and dependencies
import matplotlib.pyplot as plt
#!pip install meteostat
from meteostat import Point, Daily, Hourly


# Create names space

path_dict = cn.create_path_names_dict()                        # Create pathnames
file_names_dict = cn.create_file_names_dict()                  # Create filenames
model_dict = cn.create_model_dict()                            # Create model parameters
cf.check_folder_struture(path_dict)                            # check that all necessary folders exist, if not, create empty folders

# Preprocessing

durations:

timeseries SMARD.de : ~ 1 min

complete extraction MaStR : ~ 8 h

additional joining operator information to MaStR extraction : ~ 8 h

## time series of SMARD.de

In [ ]:
#### --- extract data from MaStR - smard.de ---
cf.preprocess_smard_historical_data()                   # ~ 1 min processing time

Go to your terminal, change directory and execute "python app.py" to browse raw data with dash plotly. Otherwise you can load app.py to VSCode Editor and press "Run Python File" top right.

In [ ]:
# for manual inspection in yupiter
df = cf.load_historical_gen_and_con_in_one_df()
input_year = ''

# select feature
## df.columns = {0: 'Biomasse [MW]', 1: 'Wasserkraft [MW]', 2: 'Wind Offshore [MW]', 3: Wind Onshore [MW], 4: Photovoltaik [MW], 
##               5: 'Sonstige Erneuerbare [MW]', 6: 'Kernenergie [MW]', 7: 'Braunkohle [MW]',8: 'Steinkohle [MW]', 9: 'Erdgas [MW]', 
##               10: 'Pumpspeicher [MW]', 11: 'Sonstige Konventionelle [MW]'}
input_value = df.columns[0]

# create plots
fig_1 = cf.plot_single_raw_time_serie(df, df.columns.get_loc(input_value))
energy_amount_per_year, energy_amount_per_year_per_year_and_month, energy_amount_per_year_per_day, energy_amount_per_week = cf.aggregate_energy_amounts_as_df(df, input_value)
fig_2 = cf.plot_aggregates_energy_amounts_per_year(energy_amount_per_year)
fig_3 = cf.plot_aggregates_energy_amounts_per_year_and_month(energy_amount_per_year_per_year_and_month)
fig_4 = cf.plot_aggregates_energy_amounts_per_day(energy_amount_per_year_per_day)
fig_5 = cf.plot_aggregates_energy_amounts_per_week(energy_amount_per_week)
heatmap_df_h_wd, input_year = cf.create_df_for_yearly_heatmap_hour_weekday(df, input_value, input_year)
fig_6 = cf.plot_timeseries_heatmap_hour_weekday(heatmap_df_h_wd, input_value, input_year)
heatmap_df_h_m, input_year = cf.create_df_for_yearly_heatmap_hour_month(df, input_value, input_year)
fig_7 = cf.plot_timeseries_heatmap_hours_month(heatmap_df_h_m, input_value, input_year)
fig_8, df_ldc_multiple = cf.plot_annual_load_duration_curve_multiple_years(df, input_value)

In [ ]:
fig_1
#fig_2
#fig_3
#fig_4
#fig_5
#fig_6
#fig_7
#fig_8

In [ ]:
# check installed power
df_installed_power = cf.load_installed_power_as_df()

# select feature
check_prefix = {0: 'Biomasse_[MW]', 1: 'Wasserkraft_[MW]', 2: 'Wind_Offshore_[MW]', 3: 'Wind_Onshore_[MW]', 4: 'Photovoltaik_[MW]', 
               5: 'Sonstige_Erneuerbare_[MW]', 6: 'Kernenergie_[MW]', 7: 'Braunkohle_[MW]',8: 'Steinkohle_[MW]', 9: 'Erdgas_[MW]', 
               10: 'Pumpspeicher_[MW]', 11: 'Sonstige_Konventionelle_[MW]'}
input_value = check_prefix[3]

fig_9 = cf.plot_single_installed_power(df_installed_power, input_value)
fig_9

## power plant register from MaStR

### extration of data from MaStR

The extraction of the all XML-files takes round about 8 hours. Please consider the following processing times for each generation technology and start with a small batch.

['prefix_solar_unity'] : '7 h',

['prefix_storage_unity'] : '60 mins',

['prefix_grid_Marktakteure_unity'] : '45 mins'

['prefix_combustion_unity'] : '14 mins',

['prefix_wind_unity'] : '3 mins',

['prefix_biomass_unity'] : '1 mins',

The extractions of the other categories are processed quickly.

In [ ]:
# --- extract information from xml files of MaStR

## -------- call multiple prefix_data_type -------- ##
# the prefix is the class of xml-files of MaStR-Export
process_these_prefix_data_types = {
                                    str(file_names_dict['filenames_raw']['prefix_solar_unity']),
                                    str(file_names_dict['filenames_raw']['prefix_wind_unity']),

                                    str(file_names_dict['filenames_raw']['prefix_biomass_unity']),
                                    ##str(file_names_dict['filenames_raw']['prefix_ee_other_unity']),
                                    str(file_names_dict['filenames_raw']['prefix_nuclear_generator_unity']),
                                    str(file_names_dict['filenames_raw']['prefix_power_consumer_unity']),

                                    str(file_names_dict['filenames_raw']['prefix_combustion_unity']),

                                    str(file_names_dict['filenames_raw']['prefix_water_generator_unity']),

                                    str(file_names_dict['filenames_raw']['prefix_storage_unity']),

                                    str(file_names_dict['filenames_raw']['prefix_gas_generator_unity']),
                                    str(file_names_dict['filenames_raw']['prefix_gas_storage_unity']),
                                    str(file_names_dict['filenames_raw']['prefix_gas_consumer_unity']),

                                    #str(file_names_dict['filenames_raw']['prefix_grid_Marktakteure_unity']),

                                    ##str(file_names_dict['filenames_raw']['prefix_grid_balancing_area']),
                                    ##str(file_names_dict['filenames_raw']['prefix_grid_single_grid_unity']),

                                    #str(file_names_dict['filenames_raw']['prefix_grid_roles_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_grid_value_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_grid_category_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_grid_unittypes_unity']),
                                    }

#### --- extract data from MaStR ---
for prefix_data_type in process_these_prefix_data_types:
    #cf.extract_data_from_MaStR_from_xml_to_csv(prefix_data_type)            # one fct for all kinds of unity types
    #dataframe_unity = cf.extract_data_from_MaStR_from_xml_to_csv(prefix_data_type)            # one fct for all kinds of unity types with fct export
    pass


Additionaly you can operator information to the units. The additional process duration times per prefix_data_type is comparable to these of the XML-Exports. For a first data inspection you don't need these informations.

In [ ]:
add_operator_information_of_these_prefix_data_types = {
                                    #str(file_names_dict['filenames_raw']['prefix_solar_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_wind_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_biomass_unity']),
                                    ##str(file_names_dict['filenames_raw']['prefix_ee_other_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_nuclear_generator_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_power_consumer_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_combustion_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_water_generator_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_storage_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_gas_generator_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_gas_storage_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_gas_consumer_unity']),
                                    }

# hint: prefix_solar_unity and prefix_storage_unity to be validated
for prefix_data_type in add_operator_information_of_these_prefix_data_types:
    cf.add_operator_information(prefix_data_type)
    pass

In [ ]:
# check preprocessed files MaStR
list_files_operator, list_files_without_operator = cf.check_complete_MaStR_xml_extractions()
print(f'units with operator infos : {list_files_operator}')
print(f'units without operator infos : {list_files_without_operator}')

## explore MaStR data

### Manual exploration

In [ ]:
# check extractions
prefix_data_type = file_names_dict['filenames_raw']['prefix_solar_unity']

df_MaStR = cf.load_unity_extractions_from_MaStR(prefix_data_type)
#df_MaStR

### Exploration with geopandas

In [ ]:
# load geodata information of Germany
gdf, gdff, gdf_transformed = cf.load_geojson()

In [ ]:
# select energy vector, perform aggregation and merge with geodataframe
plot_these_prefix_data_types = {
                                    #str(file_names_dict['filenames_raw']['prefix_solar_unity']),
                                    str(file_names_dict['filenames_raw']['prefix_wind_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_biomass_unity']),
                                    ##str(file_names_dict['filenames_raw']['prefix_ee_other_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_nuclear_generator_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_power_consumer_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_combustion_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_water_generator_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_storage_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_gas_generator_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_gas_storage_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_gas_consumer_unity']),
                                    }

# Select energy vector
prefix_data_type = list(plot_these_prefix_data_types)[0]
df_MaStR = cf.load_unity_extractions_from_MaStR(prefix_data_type)

# Select aggregation features
#aggregation_features = ['Bruttoleistung', 'Nettonennleistung']
class_features = 0
aggregation_feature = model_dict['aggregation_feature_per_postcode_classes_for_installed_power'][prefix_data_type][class_features]

# group feature and merge df with geodf
df_MaStR_postcode_aggregates = cf.aggregate_MaStR_extraction_per_postcode(df_MaStR, prefix_data_type, aggregation_feature)
merged_data = pd.merge(gdf, df_MaStR_postcode_aggregates, left_on='postcode', right_on='Postleitzahl', how='left')

In [ ]:
# static output:
vmax_input = 6000
cmap_input = prefix_data_type
print(prefix_data_type)
#merged_data.plot(column=aggregation_feature, legend='True', vmax=vmax_input)
merged_data.plot(column=aggregation_feature, legend='True')

In [ ]:
# interactive map
print(prefix_data_type)
fig_2 = merged_data.explore(aggregation_feature, legend='True')
fig_2

### Explore with plotly (beta)

In [ ]:
# load geodata information of Germany
gdf, gdff, gdf_transformed = cf.load_geojson()

In [ ]:
# select energy vector, perform aggregation and merge with geodataframe
plot_these_prefix_data_types = {
                                    #str(file_names_dict['filenames_raw']['prefix_solar_unity']),
                                    str(file_names_dict['filenames_raw']['prefix_wind_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_biomass_unity']),
                                    ##str(file_names_dict['filenames_raw']['prefix_ee_other_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_nuclear_generator_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_power_consumer_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_combustion_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_water_generator_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_storage_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_gas_generator_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_gas_storage_unity']),
                                    #str(file_names_dict['filenames_raw']['prefix_gas_consumer_unity']),
                                    }

# Select energy vector
prefix_data_type = list(plot_these_prefix_data_types)[0]
df_MaStR = cf.load_unity_extractions_from_MaStR(prefix_data_type)

# Select aggregation features
#aggregation_features = ['Bruttoleistung', 'Nettonennleistung']
class_features = 0
aggregation_feature = model_dict['aggregation_feature_per_postcode_classes_for_installed_power'][prefix_data_type][class_features]

# group feature and merge df with geodf
df_MaStR_postcode_aggregates = cf.aggregate_MaStR_extraction_per_postcode(df_MaStR, prefix_data_type, aggregation_feature)
merged_data = pd.merge(gdf, df_MaStR_postcode_aggregates, left_on='postcode', right_on='Postleitzahl', how='left')

In [ ]:
os.chdir(path_dict['postcodes_geodata'])
filename = 'postleitzahlen.geojson'
gdf = geopandas.read_file(filename)
gdf.to_file('main_map.geojson', driver='GeoJSON')
#gdf_transformed.to_file('main_map.geojson', driver='GeoJSON')
with open ('main_map.geojson', 'r') as infile:
    map_json = json.load(infile)

In [ ]:
fig_1 = px.choropleth_mapbox(df_MaStR_postcode_aggregates, geojson=map_json,
                             locations='Postleitzahl', featureidkey='properties.postcode',
                             color=aggregation_feature,
                             )
fig_1.show()